In [ ]:
from typing import Iterable, Literal
import pandas as pd

import networkx as nx

In [ ]:
activity = pd.read_csv(
    "data/higgs-activity_time.txt",
    sep=" ",
    header=None,
    names=["source", "target", "time", "interaction"],
)
activity["time"] = pd.to_datetime(activity["time"], unit="s")
activity["interaction"] = activity["interaction"].astype("category")

social = pd.read_csv(
    "data/higgs-social_network.edgelist",
    sep=" ",
    header=None,
    names=["source", "target"],
)

In [ ]:
activity_p1 = activity[activity["time"] < "2012-07-02 13:00:00"]
activity_p2 = activity[
    (activity["time"] >= "2012-07-02 13:00:00")
    & (activity["time"] < "2012-07-03 00:00:00")
]
activity_p3 = activity[
    (activity["time"] >= "2012-07-03 00:00:00")
    & (activity["time"] < "2012-07-04 08:00:00")
]
activity_p4 = activity[activity["time"] >= "2012-07-04 08:00:00"]

## Filter nodes

In [ ]:
def filter_nodes(data: pd.DataFrame, nodes: Iterable[int]) -> pd.DataFrame:
    """
    Filter the DataFrame to include only the specified nodes.
    """
    return data[data["source"].isin(nodes) | data["target"].isin(nodes)]

## Graph building

In [ ]:
def build_social_graph(data: pd.DataFrame) -> nx.Graph:
    """
    Build a social graph from the DataFrame.
    """
    G = nx.Graph()
    for _, row in data.iterrows():
        G.add_edge(row["source"], row["target"])
    return G


def build_activity_graph(
    data: pd.DataFrame, interaction: Literal["all", "RT", "MT", "RE"] = "all"
) -> nx.Graph:
    """
    Build an activity graph from the DataFrame.
    """
    G = nx.Graph()
    for _, row in data.iterrows():
        if interaction == "all":
            # Add the edge with interaction as attribute
            G.add_edge(row["source"], row["target"], interaction=row["interaction"])
        elif row["interaction"] == interaction:
            # Add the edge
            G.add_edge(row["source"], row["target"])
    return G